In [1]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [2]:
from nltk.sentiment import SentimentIntensityAnalyzer as sia
from nltk.stem import WordNetLemmatizer as lem

# dream = input()
dream = "Yes, my deceased spouse, alive and well. I realize in the dream that they are deceased and it's explained to me by friends also appearing in the dream that I shouldn't say anything about it as they don't remember that they died and It would upset them. It's upsetting, weird and recurring."

def dreampreprocess(dream):
  dream = ''.join([i for i in dream if i.isalpha() or i == ' '])
  dream = dream.lower()

def evaluateemotion(dream):
  emotion = sia().polarity_scores(dream)
  # 0.5 is a minimum treshold to guarantee overall mood
  if(emotion['pos']>0.5): print("Your overall mood is positive :D")
  elif(emotion['neg']>0.5): print("Your overall mood is not so positive :X")
  else: print("Your overall mood is neutral :j")
  return emotion


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
import csv
file = '/content/drive/MyDrive/Miscelleneous/CityHack2022/KeyTopic.csv'
with open(file) as file:
  content = file.readlines()
topic={}
for line in content:
  line = line.split(',')
  topic[line[0]]=[]
  for i in line[1:]:
    topic[line[0]] += [lem().lemmatize(i)]

In [4]:
def topicpresent(dream):
  dreamword = nltk.word_tokenize(dream)
  # print(dreamword)

  dreamword = [lem().lemmatize(i) for i in dreamword] 
  amount={'lengthword':len(dreamword)}
  for key in topic:
    for wordrelated in topic[key]:
      if dreamword.count(wordrelated)>0: amount[key]=amount.get(key,0)+dreamword.count(wordrelated)
  amount = sorted(amount.items(), key=lambda x: x[1], reverse=True)
  print("Your dream might be related to ", end="")
  # only show top 3
  for i in amount[1:4]:
    print(i[0][0].upper()+i[0][1:],end=" ")
  return amount

# **The following piece of code is an example when each input is fed**
**which is to say, a dream in each day**

In [5]:
# dream = input()
dream = "Yes, my deceased spouse, alive and well. I realize in the dream that they are deceased and it's explained to me by friends also appearing in the dream that I shouldn't say anything about it as they don't remember that they died and It would upset them. It's upsetting, weird and recurring."

dreampreprocess(dream)
emotion = evaluateemotion(dream)
topiccount = topicpresent(dream)

Your overall mood is neutral :j
Your dream might be related to Satisfied Pain Sad 

# **While each output is provided for users,** 
**our application keeps the record to analyze the repetition of dreams and evaluate the risk of mental illness ie. depression**

In [8]:
# emotion, topiccount variables are mentioned above

track = {}

userid = 'userid'
track[userid] = {'warning':0} #only to initialize

for topic in topiccount[1:]: #exclude whole word length
  track[userid][topic[0]] = track[userid].get(topic[0],0)+topic[1]

for warn in range (track[userid]['warning']):
  if warn>5:
    # raise questions regarding to habits of people with depression
    score = 0
    questions = ["Little interest or pleasure in doing things?","Feeling down, depressed, or hopeless?",
                 "Trouble falling or staying asleep, or sleeping too much?",
                 "Feeling tired or having little energy?",
                 "Poor appetite or overeating?",
                 "Feeling bad about yourself - or that you are a failure or have let yourself or your family down?",
                 "Trouble concentrating on things, such as reading the newspaper or watching television?",
                 "Moving or speaking so slowly that other people could have noticed?",
                 "being so fidgety or restless that you have been moving around a lot more than usual?",
                 "Thoughts that you would be better off dead, or of hurting yourself in some way?"]
    print("Over the last two weeks, how often have you been bothered by any of the following problems? (Not at all = 0, Several days = 1, More than half the days = 2, Nearly every day =3)")
    for question in questions:
      ans = int(input(question))
      score+=ans
    if (score>10): "You may have a higher likelihood of experiencing a depressive disorder than other users. We and our psychiatrists are here to help you"
    else: "Keep Going !"
    break
  elif warn>3:
    # ask user's current-state emotion to filter out the likelihood that the dream may not represent bad feeling towards such topic
    ans = input("Are you feeling okay now? (y/n")
    if ans=='y' : break #skip the whole process to add more warning
    elif ans=='n': 
      if emotion['neg']>0.5:
        i = 1
        while topiccount[i][1]/topiccount[0][1] > 0.075:
          # 0.075 or 7.5% is a value we considered as a large portion of the story
          track[userid]['warning']={topiccount[i][0]:track['warning'].get(topiccount[i],0)+1}